In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import chart_utils
import analysis_utils

In [4]:
# Download CoinMetrics data if needed

# try:
#     os.remove("btc.csv")
# except FileNotFoundError:
#     pass
# !wget https://coinmetrics.io/newdata/btc.csv

In [10]:
data = pd.read_csv("data/05_miner_herf_5_full.csv")
data = analysis_utils.get_extra_datetime_cols(data, 'date', date_format="%Y-%m-%d")
price_data = pd.read_csv("btc.csv")
data = data.merge(price_data[
    ['date', 'PriceUSD']], on='date')

In [17]:
window_sizes = [7, 14, 28, 56, 70]
y1_series = []
for window_size in window_sizes:
    data['herfindahl_index_{}_roll'.format(window_size)] = data['herfindahl_index'].rolling(window_size).mean()
    y1_series.append('herfindahl_index_{}_roll'.format(window_size))

In [34]:
chart_utils.two_axis_chart(
    data[:-28], x_series='date', 
    y1_series='herfindahl_index_28_roll',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index After Five Downstream Transactions',
    data_source="Proprietary; Price from CoinMetrics"
)

PlotlyError: The `figure_or_data` positional argument must be `dict`-like, `list`-like, or an instance of plotly.graph_objs.Figure

In [13]:
# data_week = data.groupby(['week_string'], group_keys=False, as_index=True).mean()[['herfindal_index', 'PriceUSD']].reset_index()
# data_week

In [14]:
# chart_utils.two_axis_chart(
#     data_week, x_series='week_string', y1_series='herfindal_index',
#     y2_series='PriceUSD',
#     y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
#     x_axis_title='Date',
#     y1_series_title='Herfindahl Index',
# #     y1_series_axis_format='{:.2f}',
#     title='Coinbase Output Herfindahl Index After Five Downstream Transactions',
#     data_source="Proprietary; Price from CoinMetrics"
# )

In [7]:
data['miner_herf_roll_100'] = data['herfindal_index'].rolling(100).mean()

In [ ]:
data

In [30]:
curves = pd.read_csv("data/05_miner_herf_4_curve_2020.csv")
curves['halving_status'] = ['pre-halving' if x < '2020-05-11' else 'post-halving' if x > '2020-05-11' else 'halving week' for x in curves['week_date']]

In [32]:
# curves['week_date'].unique()
curves['halving_status'].unique()

array(['pre-halving', 'halving week', 'post-halving'], dtype=object)

In [14]:
curves.loc[curves['week_date'] == '2020-02-10'].reset_index(drop=True)

,date_agg,week_date,days_since_coinbase,herfindal_index
0,202027,2020-02-10,0,0.991306
1,202027,2020-02-10,1,0.657386
2,202027,2020-02-10,2,0.304118
3,202027,2020-02-10,3,0.226284
4,202027,2020-02-10,4,0.202921
...,...,...,...,...
137,202027,2020-02-10,137,0.161581
138,202027,2020-02-10,138,0.160057
139,202027,2020-02-10,139,0.156437
140,202027,2020-02-10,140,0.133571


In [ ]:
'2020-05-11'

In [27]:
chart_utils.single_axis_chart(
    curves.loc[curves['week_date'] == '2020-03-23'].reset_index(drop=True),
    x_series='days_since_coinbase', y1_series='herfindal_index',
    x_axis_title='Days Since Coinbase',
    y1_series_title='Coinbase Output Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index Curve: 4 deep',
    data_source="Bitcoin ETL"
)

In [29]:
chart_utils.miner_herf_chart(
    curves,
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='log',
    y1_series_axis_range=[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve: 4 deep',
    data_source="Bitcoin ETL"
)

In [ ]:
# data5 = pd.read_csv("data/05_miner_herf_5.csv")
# data5['miner_herf_roll_100'] = data5['herfindal_index'].rolling(100).mean()

# data3 = pd.read_csv("data/05_miner_herf_3.csv")
# data3['miner_herf_roll_100'] = data3['herfindal_index'].rolling(100).mean()
# merge = data5.merge(data3, on='coinbase_block_number', suffixes=('5', '3'))
# merge['miner_herf_roll_1003'] - merge['miner_herf_roll_1005']
# merge[['coinbase_block_ts5', 'coinbase_block_number', 'miner_herf_roll_1003', 'miner_herf_roll_1005']]

In [ ]:
chart_utils.single_axis_chart(
    data, x_series='coinbase_block_number', y1_series='miner_herf_roll_100',
    x_axis_title='Block Number',
    y1_series_title='Coinbase Output Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index: 5 deep',
    data_source="Bitcoin ETL"
)

In [ ]:
chart_utils.single_axis_chart(
    data, x_series='coinbase_block_number', y1_series='miner_herf_roll_100',
    x_axis_title='Block Number',
    y1_series_title='Coinbase Output Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index: 4 deep',
    data_source="Bitcoin ETL"
)

In [ ]:
chart_utils.single_axis_chart(
    data, x_series='coinbase_block_number', y1_series='miner_herf_roll_100',
    x_axis_title='Block Number',
    y1_series_title='Coinbase Output Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index: 3 deep',
    data_source="Bitcoin ETL"
)